In [2]:
import yfinance as yf
from datetime import datetime, timedelta
from prettytable import PrettyTable
import os
from pandas_market_calendars import get_calendar
import pandas as pd
import re
import time


# Define the path to the CSV file containing 'Client Name'
csv_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Individual_Investor_Data\Individual_Investor_Reconciliation\Individual_Investor_LIST_240130160546.csv'

# Read 'Client Name' from the CSV file
client_name_df = pd.read_csv(csv_path)
client_names = client_name_df['Client Name'].tolist()

# Define the path to the folder containing Excel files
excel_folder_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\OutboundSuperstars'

# Define the start and end dates for the file date range
start_date = datetime.strptime('20230601', "%Y%m%d")
end_date = datetime.strptime('20240201', "%Y%m%d")

# Create a PrettyTable to store the results
results_table = PrettyTable()
results_table.field_names = ['Code', 'Client Name', 'Holding', 'Percent Holding', 'Investor Type', 'File Date']

# Create a list to store matching rows
matching_rows = []

# Iterate through each Excel file
for excel_file in os.listdir(excel_folder_path):
    excel_file_path = os.path.join(excel_folder_path, excel_file)

    # Get the list of sheet names in the Excel file
    sheet_names = pd.ExcelFile(excel_file_path).sheet_names

    # Check if 'Shareholding' sheet exists
    if 'Shareholding' in sheet_names:
        # Read the 'Shareholding' sheet
        df = pd.read_excel(excel_file_path, sheet_name='Shareholding', skiprows=6)

        # Extract date from the file name using regular expression
        match = re.search(r'\d{8}', excel_file)  # Assuming the date in the file name is in YYYYMMDD format
        if match:
            file_date_str = match.group(0)
            file_date = datetime.strptime(file_date_str, "%Y%m%d")

            # Check if the file date is within the specified range
            if start_date <= file_date <= end_date:
                # Check if 'Client Name' is present in the DataFrame
                if 'Client Name' in df.columns:
                    # Check if each 'Client Name' from the CSV is present in the Excel file
                    present_in_excel = [client_name in df['Client Name'].values for client_name in client_names]

                    # Display only the specified columns for matched records
                    for i, present in enumerate(present_in_excel):
                        if present:
                            matching_row = df[df['Client Name'] == client_names[i]]
                            matching_row = matching_row[['Code', 'Client Name', 'Holding', 'Percent Holding', 'Investor Type']]
                            matching_row['File Date'] = file_date_str
                            matching_rows.append(matching_row)

# Concatenate all matching rows into a DataFrame
results_df = pd.concat(matching_rows)

# Sort the DataFrame by 'File Date' in descending order
results_df = results_df.sort_values(by='File Date', ascending=False)

# Print the results using PrettyTable
for _, row in results_df.iterrows():
    results_table.add_row(row.tolist())



from datetime import datetime, timedelta

def is_working_day(date_str):
    # Parse the date string to datetime format
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Check if the date falls on a weekend (Saturday or Sunday)
    if date_obj.weekday() >= 5:
        return False
    
    # Check if the date falls on a special working day
    special_working_days = {"2024-01-27"}  # Example: Christmas Eve
    if date_str in special_working_days:
        return True
    
    # Check if the date falls on a holiday
    # Add your holiday logic here
    holidays = {"2023-01-26", "2023-03-07", "2023-03-30", "2023-04-04", "2023-04-07", 
                "2023-04-14", "2023-05-01", "2023-06-29", "2023-07-29", "2023-08-15", 
                "2023-09-19", "2023-10-02", "2023-10-24", "2023-11-14", "2023-11-27", 
                "2023-12-25","2024-01-22",
                "2024-01-26", "2024-03-08", "2024-03-25", "2024-03-29", "2024-04-11", 
                "2024-04-17", "2024-05-01", "2024-06-17", "2024-07-17", "2024-08-15", 
                "2024-10-02", "2024-11-01", "2024-11-15", "2024-12-25"}
    if date_str in holidays:
        return False
    
    # If the date is not a weekend, holiday, or special working day, it's a regular working day
    return True

def is_working_day_if_not_next_working_date(date):
    # Check if the input is already a string
    if isinstance(date, str):
        # Parse the date string to datetime format
        date_obj = datetime.strptime(date, '%Y-%m-%d')
    else:
        # Use the provided Timestamp object
        date_obj = date
    
    # Check if the date is a working day
    if is_working_day(date_obj.strftime('%Y-%m-%d')):
        return date_obj
    
    # If the date is not a working day, find the next working date
    while True:
        date_obj += timedelta(days=1)
        if is_working_day(date_obj.strftime('%Y-%m-%d')):
            return date_obj




def download_stock_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

def calculate_percentage_changes(data, stock_code, start_date,client_name,holding,percent_holding,investor_type,file_date ):
    result_dict = {
        "Stock Code": stock_code,
        "Client Name": client_name,  # Corrected variable name
        "Holding": holding,  # Corrected variable name
        "Percent Holding": percent_holding,  # Corrected variable name
        "Investor Type": investor_type,  # Corrected variable name
        "File Date": file_date,  # Corrected variable name
    }


    periods = [1, 20, 40, 60]

    for period in periods:
         if len(data) >= period + 1:

            close_change_1d = volume_change = None
    
            # Check if the start date is in the index
            if start_date in data.index:
                close_change_1d = ((data['Close'].iloc[period] - data['Close'].loc[start_date]) / data['Close'].loc[start_date]) * 100
                volume_change = ((data['Volume'].iloc[period] - data['Volume'].loc[start_date]) / data['Volume'].loc[start_date]) * 100
    
            result_dict[f"{period}D P (%)"] = f"{close_change_1d:.2f}" if close_change_1d is not None else "nan"
            result_dict[f"{period}D V (%)"] = f"{volume_change:.2f}" if volume_change is not None else "nan"

    return result_dict



# Assuming results_df has a 'File Date' column in '%Y%m%d' format
results_df['File Date'] = pd.to_datetime(results_df['File Date'], format='%Y%m%d')



# Create a PrettyTable for displaying the results
table = PrettyTable()
header = ["Stock Code", "Client Name","Holding","Percent Holding","Investor Type","File Date"]

for period in [1, 20, 40, 60]:
    header.append(f"{period}D P (%)")
    header.append(f"{period}D V (%)")
table.field_names = header

# Create a raw table to display original data
raw_table = PrettyTable()
raw_table.field_names = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "Stock Code"]



for _, row in results_df.iterrows():
    
    # Assuming results_df has a 'File Date' column in '%Y%m%d' format
    file_date = pd.to_datetime(row['File Date'], format='%Y%m%d')
    
    # Define start date
    start_date = file_date  # Select the file date as the start date

    # while True:
    #     # Find the next day
    start_date_pass = is_working_day_if_not_next_working_date(start_date)
    print(start_date_pass)
    # Calculate end date for historical data (two months later)
    end_date = (start_date_pass + timedelta(days=120)).strftime("%Y-%m-%d")

    # Download stock data for the next day
    if isinstance(row['Code'], str):
        stock_data = download_stock_data(row['Code'] + ".NS", start_date_pass, end_date)
  
        percentage_changes = calculate_percentage_changes(stock_data, row['Code'], start_date_pass, row['Client Name'], row['Holding'], row['Percent Holding'], row['Investor Type'], file_date)
    
        # Add the result to the PrettyTable
        table.add_row([percentage_changes.get(field, "nan") for field in table.field_names])


print("\nPercentage Changes:")
print(table)



2024-01-23 00:00:00
2024-01-23 00:00:00
[*********************100%%**********************]  1 of 1 completed

C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



2024-01-23 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-01-23 00:00:00
[*********************100%%**********************]  1 of 1 completed

C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')



2024-01-23 00:00:00
[*********************100%%**********************]  1 of 1 completed
2024-01-23 00:00:00
[*********************100%%**********************]  1 of 1 completed

...


2023-07-05 00:00:00
[*********************100%%**********************]  1 of 1 completed
2023-07-05 00:00:00
[*********************100%%**********************]  1 of 1 completed
2023-07-04 00:00:00


C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


[*********************100%%**********************]  1 of 1 completed
2023-07-04 00:00:00
[*********************100%%**********************]  1 of 1 completed
2023-07-04 00:00:00
[*********************100%%**********************]  1 of 1 completed
2023-06-30 00:00:00
2023-06-30 00:00:00
2023-06-01 00:00:00
[*********************100%%**********************]  1 of 1 completed

C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\91908\AppData\Roaming\Python\Python39\site-packages\yfinance\utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')

1 Failed download:
['526703.NS']: Exception('%ticker%: No timezone found, symbol may be delisted')




Percentage Changes:
+------------+-----------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
| Stock Code |                  Client Name                  |    Holding     | Percent Holding | Investor Type |      File Date      | 1D P (%) | 1D V (%) | 20D P (%) | 20D V (%) | 40D P (%) | 40D V (%) | 60D P (%) | 60D V (%) |
+------------+-----------------------------------------------+----------------+-----------------+---------------+---------------------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+
| AVADHSUGAR |         Anil Kumar Goel and Associates        | Sold -0.01000  |       6.89      |   Individual  | 2024-01-20 00:00:00 |   0.63   |  -35.63  |    nan    |    nan    |    nan    |    nan    |    nan    |    nan    |
|  KRISHANA  |                Ashok Kumar Jain            